In [1]:
import numpy as np
from scipy.ndimage.filters import uniform_filter1d
import random
import matplotlib.pyplot as plt

In [2]:
class EM:
    def __init__(self,Y_M, Y_N = None, Z_N = None, sigma_e_init = 0, W_init = 0 ):
        self.Y_N = Y_N
        self.Y_M = Y_M
        self.Z_N = Z_N
        self.Z_M = None
        
        
        self.M = len(Y_M)
        self.N = len(Y_N)  if type(self.Y_N) != type(None) else 0
        self.S = self.M + self.N
        
        self.sigma_e = sigma_e_init
        self.W = W_init
        
        self.EZ = None
        self.EZZ = None
        
        
    def e_step(self):
        A = (self.W**2) * (self.sigma_e**-2) + 1
        B = self.W*self.Y_M*(self.sigma_e**-2)
        
        self.EZ = B/A
        self.EZZ = (1/A)  + (B/A)**2
        
    def m_step(self):
        # Must update W first, as it is required for sigma
        self.W = self.new_W()
        self.sigma_e = self.new_sigma_e()
        
        
    def new_sigma_e(self):
        _1 = np.sum((self.Y_N-W*self.Z_N)**2) if type(self.Y_N) != type(None) else 0
        _2 = np.sum(self.Y_M**2)
        _3 = -2*self.W*np.sum(self.Y_M*self.EZ)
        _4 = self.W**2 * np.sum(self.EZZ)

        return np.sqrt((_1 + _2 + _3 + _4)/self.S)
    
    
    def new_W(self):
        _1 = np.sum(self.Y_N*self.Z_N)  if type(self.Y_N) != type(None) else 0
        _2 = np.sum(self.Y_M*self.EZ)
        _3 = np.sum(self.Z_N**2) if type(self.Y_N) != type(None) else 0
        _4 = np.sum(self.EZZ)
        return (_1 + _2)/(_3 + _4)

        

In [3]:
def test(filter_size, sigma_e, W, ratio_obs, sigma_init, W_init):
    size = 10000
    num_obs = int(size*ratio_obs)

    Z = np.random.normal(0,1,size)
    e = np.random.normal(0, sigma_e, size)

    Y = W*Z + e
    
    # Complete data solution:
    C_W = np.sum(Y*Z)/np.sum(Z**2)
    C_sigma_e = np.sqrt(np.sum((Y-C_W*Z)**2)/size)

    print(f'Complete Data solution: (sig_e, W): {C_sigma_e, C_W}')
    
    Y_N_indexes = random.sample(range(size), num_obs)
    Y_M_indexes = [i for i in range(size) if i not in Y_N_indexes]
    
    Y_M = Y[Y_M_indexes]
    Y_N = Y[Y_N_indexes]
    
    filter_ = uniform_filter1d(Y, size=filter_size, mode='constant')
    Z_N = filter_[Y_N_indexes]/W
    

    EM1 = EM(Y_M, Y_N, Z_N, sigma_init, W_init)
    for i in range(100):
        EM1.e_step()
        EM1.m_step()
        print(EM1.sigma_e, EM1.W)
    
    print('Z mean SQ difference:', np.mean((Z_N - Z[Y_N_indexes])**2))

In [4]:
sigma_e = 0.05
W = 5
ratio_obs = 0.25

# filter_size = 3

In [5]:
filter_size = 3
test(filter_size, sigma_e, W, ratio_obs, sigma_e, W)

Complete Data solution: (sig_e, W): (0.049750619664001454, 5.000130244091275)
1.9901883784603092 5.027213034197315
2.6184806182265348 4.944237631519934
2.968976034496857 4.73752741380668
3.1793000567271874 4.503249356156665
3.31666984145535 4.301204332119113
3.4135976573473927 4.145017643032286
3.4852294506767736 4.028497756873496
3.5392591989353903 3.94185027583622
3.580329323848574 3.876933019319088
3.611653498061691 3.8278761949349116
3.635597816628576 3.790552175597589
3.6539368770845426 3.762018825699261
3.668007960588546 3.7401343792961135
3.6788212569432317 3.7233116944247
3.687141991384448 3.7103595445183553
3.6935516436383806 3.7003760677225968
3.698493435935811 3.6926744359753934
3.7023061489425695 3.6867294403842226
3.7052493494548293 3.682138293777693
3.7075223035937386 3.6785914460998725
3.7092782290144473 3.6758506341876696
3.7106350860605195 3.673732253871051
3.711683782311792 3.672094698461527
3.712494432269667 3.670828680763288
3.7131211468422336 3.6698498146425256
3.7

# filter_size = 5

In [6]:
filter_size = 5
test(filter_size, sigma_e, W, ratio_obs, sigma_e, W)

Complete Data solution: (sig_e, W): (0.04975980463861455, 5.0005762465293415)
2.3032906090673997 4.980252420144053
3.023373428274184 4.795556701372833
3.4111875450255678 4.4500967230615975
3.6414041871115583 4.103496062184123
3.7967160661990316 3.8250195308814496
3.9109878824734454 3.614151552171644
3.9979740821839727 3.4537637702647244
4.064966300697313 3.3295046242895254
4.116937206437627 3.2317578259084696
4.157545479295206 3.154076299961375
4.189498625787787 3.091917080286695
4.214801967950981 3.0419377645139662
4.234950472463103 3.0016077423225007
4.251069677698447 2.968974815779519
4.2640160931767035 2.9425133810866746
4.274448231462481 2.9210198737844646
4.282877131605708 2.90353792804529
4.289702675963487 2.889303323095978
4.295240021782593 2.8777026528160157
4.299739086699829 2.8682418037441617
4.303399094229863 2.8605216091114727
4.306379560326234 2.854218852356171
4.308808682384667 2.8490713203577784
4.310789808104372 2.8448659619593717
4.31240646763773 2.8414294533013553
4.

# filter_size = 10

In [7]:
filter_size = 10
test(filter_size, sigma_e, W, ratio_obs, sigma_e, W)

Complete Data solution: (sig_e, W): (0.0494375744184835, 5.00030931504548)
2.3531820897104594 5.0012336246474005
3.088827474685435 4.831205776467047
3.482293018378691 4.472213271703404
3.7150862554372113 4.1004876641613555
3.873972606082123 3.7974109747745053
3.993326452311902 3.5646376458341673
4.086204770259232 3.3842081888452737
4.159312185173854 3.2410887061153733
4.217319343924483 3.125432437320042
4.263744517641299 3.0307690176860476
4.3012268971534064 2.95259021726722
4.331737259993291 2.8875918040926862
4.356754340012735 2.8332678338958384
4.37739867853704 2.787673189728903
4.394529256198164 2.749272335787336
4.408812269010271 2.7168370911902593
4.420770168190783 2.6893746729787136
4.430816919942163 2.6660755330827848
4.439283634203259 2.6462746745449675
4.446437412804314 2.6294224291128083
4.452495376297193 2.6150620365247894
4.457635224883542 2.602812210652233
4.4620032796200695 2.5923534237189734
4.46572067111776 2.583417003537524
4.468888151206351 2.575776386927579
4.471589

# filter_size = 15

In [8]:
filter_size = 15
test(filter_size, sigma_e, W, ratio_obs, sigma_e, W)

Complete Data solution: (sig_e, W): (0.05007365304892752, 5.000160286737924)
2.47117156556239 4.999135864987103
3.235193059666108 4.765948469058643
3.6327000992611707 4.3226660904067415
3.8669227396914656 3.8915595447518516
4.029892457200494 3.551007396759114
4.153939094304747 3.290032630236898
4.250738739838782 3.0848179091799754
4.3269639668185675 2.9188707102720044
4.387588585309551 2.7820529673100687
4.436357109695706 2.667750861440301
4.476032330102538 2.571321654895115
4.50864694051797 2.4893414486447636
4.5357086892281115 2.419203938943031
4.558349618763212 2.3588804069573657
4.577430902831446 2.306764354201782
4.593616174897931 2.2615656867282476
4.607423164496537 2.222236077678271
4.61926045028597 2.1879149285657746
4.6294539372725545 2.157889424994683
4.6382661676740895 2.1315644977973793
4.6459105763532955 2.10843989647114
4.652562137486911 2.088092461468159
4.658365404795633 2.070162255492303
4.663440647761781 2.0543415982537963
4.667888581727767 2.040366312388176
4.6717940

# filter_size = 20

In [9]:
filter_size = 20
test(filter_size, sigma_e, W, ratio_obs, sigma_e, W)

Complete Data solution: (sig_e, W): (0.04973508987229002, 4.999873671815734)
2.3683342171976167 4.999837983396929
3.1090172806022984 4.8270091824772905
3.5051878092521838 4.457253657755399
3.740461679703911 4.071905766792334
3.9027108646913744 3.7558866680908554
4.026177772769472 3.510944773786528
4.12347935201026 3.3185233272302415
4.201046898570172 3.1633476008178736
4.263435805740452 3.0355920532371004
4.31411737734784 2.928884845305471
4.355708006160555 2.8388208515932463
4.390165632572452 2.7621858012588274
4.418961533377936 2.6965478229650426
4.443212212568685 2.6400206007122335
4.463774966518243 2.5911135679208455
4.481316019741041 2.548631328536178
4.496359161915747 2.511603338860222
4.509320742133793 2.479233284185806
4.520535111670177 2.450861799810664
4.530273331996181 2.4259385090956416
4.538757087743027 2.4040007217167956
4.546169148959854 2.384656985485571
4.552661322866624 2.3675742332223493
4.55836055937328 2.352467630368489
4.563373684642221 2.3390924770253254
4.567791

# filter_size = 25

In [10]:
filter_size = 25
test(filter_size, sigma_e, W, ratio_obs, sigma_e, W)

Complete Data solution: (sig_e, W): (0.05053641277961426, 4.999926236059463)
2.434370464296677 4.998273843683268
3.19339585817601 4.806796952154398
3.595034546430583 4.406789283243154
3.8326913792376605 3.998396937647132
3.997468128880376 3.667200997314401
4.123290736655205 3.4105057449799077
4.222407537579312 3.2075651504199856
4.301333526105387 3.0426021447488663
4.3648055035675055 2.9057153508474536
4.41642012385376 2.790484145756818
4.458863149607123 2.6924437790705613
4.4941294766007935 2.6083229495888367
4.5237100083947634 2.535641450477158
4.548731357691915 2.4724735364280157
4.570055526954987 2.417296608329946
4.588350338554262 2.3688889778404887
4.604139407723607 2.3262581511205465
4.617837936165902 2.2885891155125275
4.629778640755231 2.2552062372479704
4.640230758937654 2.2255446867283886
4.649414143152882 2.1991286844301383
4.657509832037847 2.1755547208125483
4.664668065155553 2.154478461125671
4.67101442205393 2.135604417814575
4.676654570402228 2.1186777270387744
4.68167